In [1]:
import os
os.chdir("../../../")
import pandas as pd
import matplotlib.pyplot as plt

from src.google_trends import GT
from dotenv import load_dotenv
load_dotenv()

from bokeh.plotting import figure, show, output_notebook, output_file
from bokeh.models import CustomJS, Select, ColumnDataSource, HoverTool, BoxZoomTool, ResetTool, Legend, WheelZoomTool
from bokeh.palettes import Category20
output_notebook()

import warnings
warnings.filterwarnings("ignore")

Loading BokehJS ...

In [2]:
GoogleAPIkey = os.getenv("GoogleAPIkey")

In [3]:
gt = GT(GoogleAPIkey)

In [4]:
trends_df = pd.DataFrame()
for term in ["economy", "inflation", "unemployment"]:
    search_term = "solomon islands " + str(term)
    trend_response = gt.get_health_trends(search_term)
    df = gt.to_df(trend_response)
    df = df.drop("term", axis=1).rename({"value": term}, axis=1)
    if trends_df.empty:
        trends_df = df
    else:
        trends_df = trends_df.merge(df, how="left", on="date")

In [5]:
local_trends_df = pd.DataFrame()
for term in ["economy", "inflation", "unemployment", "job"]:
    local_response = gt.get_graph(term, "SB")
    local_response_df = gt.to_df(local_response)
    local_response_df = (local_response_df.drop("term", axis=1)
                            .rename({"value": term}, axis=1))
    if local_trends_df.empty:
        local_trends_df = local_response_df
    else: 
        local_trends_df = local_trends_df.merge(local_response_df, how="left", on="date")

In [13]:
output_file(filename=os.getcwd() +
            "/data/text/solomon_islands/solomon_islands_local_trends.html",
            title="Solomon Islands Google Trends")

hover = HoverTool(tooltips=[('Date', '@date{%Y-%m-%d}'),
                            ('Number', '@$name')],
                  formatters={'@date': 'datetime'})
p = figure(height=400,
           width=700,
           title="Google Trend Index By Month (search conducted in Solomon Islands)",
           x_axis_type="datetime",
           x_axis_label="Date",
           y_axis_label="Google Trend Index",
           tools=[hover, BoxZoomTool(), ResetTool(), WheelZoomTool()])

cols = [col for col in local_trends_df.columns if col != "date"]
p.add_layout(Legend(), 'right')
source = ColumnDataSource(trends_df)

for col, color in zip(cols, Category20[3]):
    # have to use different colnames for y-coords so tooltip can refer to @$name
    p.line('date',
           col,
           source=local_trends_df,
           name=col,
           legend_label=col,
           color=color)

p.legend.label_text_font_size = '9pt'
p.legend.click_policy = "mute"
p.legend.location = "top_left"

show(p)

In [14]:
output_file(filename=os.getcwd() +
            "/data/text/solomon_islands/solomon_islands_econ_trends.html",
            title="Solomon Islands Google Trends")

hover = HoverTool(tooltips=[('Date', '@date{%Y-%m-%d}'),
                            ('Number', '@$name{.3f}')],
                  formatters={'@date': 'datetime'})
p = figure(height=400,
           width=700,
           title="Google Trend Index By Month",
           x_axis_type="datetime",
           x_axis_label="Date",
           y_axis_label="Google Trend Index",
           tools=[hover, BoxZoomTool(), ResetTool()])

cols = [col for col in trends_df.columns if col != "date"]
p.add_layout(Legend(), 'right')
source = ColumnDataSource(trends_df)

for col, color in zip(cols, Category20[3]):
    # have to use different colnames for y-coords so tooltip can refer to @$name
    p.line('date',
           col,
           source=trends_df,
           name=col,
           legend_label=col,
           color=color)

p.legend.label_text_font_size = '9pt'
p.legend.click_policy = "mute"
p.legend.location = "top_left"

show(p)